In [ ]:
from wholeslidedata.annotation.wholeslideannotation import WholeSlideAnnotation
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from wholeslidedata.annotation.utils import plot_annotations
from matplotlib import pyplot as plt
from pprint import pprint
from wholeslidedata.annotation import utils as annotation_utils
import numpy as np
from PIL import Image, ImageDraw, ImageFont

import sys
import os
sys.path.append("C:/Users/mbotros/ASAP2.0/bin/")
from wholeslidedata.accessories.asap.imagewriter import WholeSlideMaskWriter

In [ ]:
def write_mask(wsi, wsa, spacing, tile_size=1024, suffix="_biopsy_mask_color.tif"):
    shape = wsi.shapes[wsi.get_level_from_spacing(spacing)]
    ratio = wsi.get_downsampling_from_spacing(spacing)
    write_spacing = wsi.get_real_spacing(spacing)

    mask_output_path = str(wsa.path).replace(".xml", suffix)

    wsm_writer = WholeSlideMaskWriter()
    wsm_writer.write(
        path=mask_output_path,
        spacing=write_spacing,
        dimensions=(shape[0], shape[1]),
        tile_shape=(tile_size, tile_size),
    )
    
    # make an image same size as the WSI
    mask = Image.new(mode='P', size=shape)
    
    # declare font
    font = ImageFont.truetype("arial.ttf", size=1500)
    
    # draw rectangle for every biopsy-outlines annotation
    for i, annotation in enumerate(wsa.annotations):
        
        bottom_left, top_right = tuple(annotation.coordinates[0]), tuple(annotation.coordinates[2])
        ImageDraw.Draw(mask).rectangle(xy=[bottom_left, top_right], width=175, outline=i + 1)
        # ImageDraw.Draw(mask).text(xy=bottom_left, text=" {}".format(i + 1), font=font, fill=1)
    
    # make it a numpy array
    mask = np.asarray(mask).T
       
    # write output 
    for y_pos in range(0, shape[1], tile_size):
        for x_pos in range(0, shape[0], tile_size):
            mask_tile = mask[x_pos: x_pos + tile_size, y_pos: y_pos + tile_size].T
            if np.any(mask_tile):
                wsm_writer.write_tile(tile=mask_tile, coordinates=(int(x_pos), int(y_pos)))

    print("closing...")
    wsm_writer.save()
    print("done")

In [ ]:
input_folder = 'C:/Users/mbotros/PhD/data/ASL/'

image_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if 'tif' in f and 'biopsy' not in f])
annotation_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if 'xml' in f])

print(len(image_files))
print(len(annotation_files))

In [ ]:
# define wsi and wsa (only take the biopsy outlines)

for path_to_wsi, path_to_wsa in zip(image_files, annotation_files):
    print('Image: {}'.format(path_to_wsi))
    print('Anno: {}'.format(path_to_wsa))
    
    wsi = WholeSlideImage(path_to_wsi, backend='asap')
    wsa = WholeSlideAnnotation(path_to_wsa, labels=['Biopsy-Outlines'])

    write_mask(wsi=wsi, wsa=wsa, spacing=0.25)
    print('\n')